## Import libraries

In [1]:
from network_optimizer import NetworkOptimizer
import synthetic_data_generators as sdg
import numpy as np
import pandas as pd

import time

## Import Synthetic Data

In [2]:
synth_network = pd.read_csv("../data/synth_network.csv")
synth_pool = pd.read_csv("../data/synth_pool.csv")
synth_members = pd.read_csv("../data/synth_members.csv")
synth_reqs = sdg.synth_reqs

In [3]:
print(f"Network size: {len(synth_network)}")
print(f"Pool size: {len(synth_pool)}")
print(f"Number of members: {len(synth_members)}")
synth_reqs

Network size: 100
Pool size: 100
Number of members: 1000


,specialty,county,provider_count,distance_req,min_access_pct,min_providers
0,cardiologist,wayne,1,15,90,5
1,pcp,wayne,2,15,90,10
2,ent,wayne,1,15,90,5
3,urologist,wayne,1,15,90,5
4,obgyn,wayne,1,15,90,5


## Run Optimization

### Create Network from scratch

In [4]:
optimizer = NetworkOptimizer(pool=synth_pool, 
                             members=synth_members,
                             adequacy_reqs=synth_reqs,
                             #no network
                             )

In [5]:
optimizer.optimize(10)

Optimization round 1 ...
Optimization round 2 ...
Optimization round 3 ...
Optimization round 4 ...
Optimization round 5 ...
Optimization round 6 ...
Optimization round 7 ...
Optimization round 8 ...
Optimization round 9 ...
Optimization round 10 ...
No more options for optimization
Average seconds per round of optimization: 0.8
Adequacy score for best network: 0.97



In [6]:
optimizer.adequacy_detail

,specialty,county,provider_count,distance_req,min_access_pct,min_providers,total_members,members_with_access,servicing_providers,pct_with_access,adequacy_index
0,cardiologist,wayne,1,15,90,5,1000,1000,11,1.000,1.000
1,pcp,wayne,2,15,90,10,1000,990,10,0.990,0.990
2,ent,wayne,1,15,90,5,1000,1000,8,1.000,1.000
3,urologist,wayne,1,15,90,5,1000,910,7,0.910,0.910
4,obgyn,wayne,1,15,90,5,1000,968,6,0.968,0.968


In [7]:
optimizer.move_tracker

[('addition', 4),
 ('addition', 2),
 ('addition', 14),
 ('addition', 0),
 ('addition', 10),
 ('addition', 13),
 ('addition', 5),
 ('addition', 9),
 ('addition', 11)]

In [8]:
optimizer.performance_history

array([0.     , 0.25128, 0.4768 , 0.65592, 0.74724, 0.80772, 0.87188,
       0.94692, 0.9714 , 0.9736 ])

In [9]:
optimizer.best_network.loc[optimizer.best_network.group_id == 4]

,npi,specialty,group_id,efficiency,effectiveness,location_id,county,latitude,longitude,pct_serving
0,72,cardiologist,4,5,1,6,wayne,42.000698,-83.751425,0.334
1,56,urologist,4,1,1,7,wayne,42.199648,-83.761273,0.628
2,89,cardiologist,4,1,1,28,wayne,42.153219,-83.988858,0.323
3,27,ent,4,1,2,45,wayne,42.286955,-83.598814,0.587
4,81,cardiologist,4,4,1,46,wayne,42.296682,-83.854589,0.548
5,72,cardiologist,4,5,1,48,wayne,42.290157,-83.631864,0.629
6,1,ent,4,2,1,49,wayne,41.938914,-83.417463,0.116
7,1,ent,4,2,1,53,wayne,42.157524,-83.947248,0.382
8,27,ent,4,1,2,54,wayne,42.000392,-83.790992,0.316
9,55,pcp,4,2,4,57,wayne,42.266743,-83.822039,0.594


### Create Network from scratch using custom objective

In [10]:
def user_objective(self, network):
    if len(network) == 0:
        return 0
    else:
        return self.adequacy(network) * network.effectiveness.mean()

In [11]:
optimizer = NetworkOptimizer(pool=synth_pool, 
                             members=synth_members,
                             adequacy_reqs=synth_reqs,
                             # no network
                             user_objective=user_objective
                             )

In [12]:
optimizer.optimize(10)

Optimization round 1 ...
Optimization round 2 ...
Optimization round 3 ...
Optimization round 4 ...
Optimization round 5 ...
Optimization round 6 ...
Optimization round 7 ...
Optimization round 8 ...
Optimization round 9 ...
Optimization round 10 ...
No more options for optimization
Average seconds per round of optimization: 0.7
Adequacy score for best network: 0.97



In [13]:
optimizer.adequacy_detail

,specialty,county,provider_count,distance_req,min_access_pct,min_providers,total_members,members_with_access,servicing_providers,pct_with_access,adequacy_index
0,cardiologist,wayne,1,15,90,5,1000,1000,8,1.000,1.000
1,pcp,wayne,2,15,90,10,1000,966,10,0.966,0.966
2,ent,wayne,1,15,90,5,1000,1000,8,1.000,1.000
3,urologist,wayne,1,15,90,5,1000,902,7,0.902,0.902
4,obgyn,wayne,1,15,90,5,1000,968,7,0.968,0.968


In [14]:
optimizer.move_tracker

[('addition', 2),
 ('addition', 1),
 ('addition', 14),
 ('addition', 0),
 ('addition', 5),
 ('addition', 13),
 ('addition', 11),
 ('addition', 10),
 ('addition', 6)]

In [15]:
optimizer.performance_history

array([0.        , 0.62345   , 1.25028471, 1.866864  , 2.31783568,
       2.54961143, 2.74885333, 2.94145333, 3.00906667, 3.03274576])

In [16]:
optimizer.best_network.loc[optimizer.best_network.group_id == 2]

,npi,specialty,group_id,efficiency,effectiveness,location_id,county,latitude,longitude,pct_serving
0,14,urologist,2,5,1,0,wayne,41.916604,-83.793571,0.175
1,5,urologist,2,2,5,4,wayne,42.189181,-83.655711,0.639
2,32,cardiologist,2,3,3,12,wayne,41.982906,-83.838536,0.268
3,62,urologist,2,2,2,22,wayne,42.113551,-83.472148,0.345
4,57,pcp,2,5,3,30,wayne,42.027369,-83.644040,0.359
5,5,urologist,2,2,5,39,wayne,42.361643,-83.532102,0.436
6,76,ent,2,5,5,63,wayne,42.396234,-83.950313,0.321
7,37,cardiologist,2,1,5,74,wayne,42.222695,-83.420171,0.356
8,37,cardiologist,2,1,5,78,wayne,42.061799,-83.649882,0.433
9,57,pcp,2,5,3,83,wayne,42.052663,-83.892604,0.324
